### Soft Dedup 전 N-gram Dataframe 생성

In [2]:
import pickle

## Load pickle
with open("/opt/output/perm/analysis/ngram_result.pkl","rb") as f:
    ngram_result = pickle.load(f)

In [3]:
import pandas as pd
# 초기화
ngramdf = pd.DataFrame(columns=["N", "Ngram_syscall", "num_of_exploit"])
item = dict()
for i in range(len(ngram_result)):
    for ngram_syscall, cnt in ngram_result[i].items():
        item["N"] = i+2
        item["Ngram_syscall"] = ngram_syscall.split(",")
        item["num_of_exploit"] = cnt
        ngramdf = ngramdf.append(item, ignore_index=True)
        item = dict()
ngramdf_origin = ngramdf.copy()
ngramdf_origin

,N,Ngram_syscall,num_of_exploit
0,2,"[12, 41]",3
1,2,"[257, 5]",40
2,2,"[107, 117]",3
3,2,"[3, 103]",3
4,2,"[8, 5]",4
...,...,...,...
11350,63,"[248, 248, 41, 248, 248, 248, 54, 0, 3, 5, 9, ...",2
11351,63,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2
11352,64,"[14, 248, 248, 248, 41, 248, 248, 248, 54, 0, ...",2
11353,64,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2


In [4]:
# 모든 익스플로잇 path laod
import subprocess
import json

pathFile = subprocess.check_output(f"find /opt/output/perm/path/*.json", shell=True).decode().strip().split()
EIDList = list(map(lambda x: x.replace("/opt/output/perm/path/","").replace(".json",""), pathFile))
pathList = dict() # 익스플로잇 코드 path dictionary의 dictionary - pathList[EID][path] = path 파일 정보
for EID in EIDList:
    with open(f"/opt/output/perm/path/{EID}.json", "r") as f:
        pathList[EID] = dict()
        pathList[EID]["path"] = json.load(f)

In [5]:
# dict {EID: all path list} 만들어 주기
eid_all_path_dict = dict()
for EID, pathset in pathList.items():
    eid_all_path_dict[EID] = list()
    for userfunc, pathset in pathset['path'].items():
        if not len(pathset) == 0:
            eid_all_path_dict[EID].extend(pathset)

In [6]:
# eid_all_path_dict 저장
import pickle

with open("eid_all_path_dict.pkl","wb") as f:
    pickle.dump(eid_all_path_dict, f)

In [5]:
import numpy as np
def find_EID_with_ngram(ngram, ngram_cnt):
    EID_with_ngram  = list()
    for EID, pathset in eid_all_path_dict.items():
        if ngram_cnt == len(EID_with_ngram):
            return EID_with_ngram
        for path in pathset:
            strpath = "," + ",".join(path) + ","
            if ngram in strpath:
                EID_with_ngram.append(EID)
                break
    if not EID_with_ngram: # if no EID
        return np.nan
    else:
        return EID_with_ngram

In [6]:
# ngrma이 나오는 EID 저장
import numpy as np
ngramdf["EIDs"] = np.nan
for index, row in ngramdf.iterrows():
    strngram = "," + ",".join(row.Ngram_syscall) + ","
    ngramdf["EIDs"][index] = find_EID_with_ngram(strngram, row.num_of_exploit)
ngramdf

/tmp/ipykernel_2896765/725562346.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ngramdf["EIDs"][index] = find_EID_with_ngram(strngram, row.num_of_exploit)
/home/mini/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,N,Ngram_syscall,num_of_exploit,EIDs
0,2,"[12, 41]",3,"[3, 50541, 926]"
1,2,"[257, 5]",40,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
2,2,"[107, 117]",3,"[1397, 40871, 47170]"
3,2,"[3, 103]",3,"[47168, 47169, 47170]"
4,2,"[8, 5]",4,"[12, 3, 43127, 50541]"
...,...,...,...,...
11350,63,"[248, 248, 41, 248, 248, 248, 54, 0, 3, 5, 9, ...",2,NaN
11351,63,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2,NaN
11352,64,"[14, 248, 248, 248, 41, 248, 248, 248, 54, 0, ...",2,NaN
11353,64,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2,NaN


In [7]:
# N-gram 처리 (시스템콜 아닌 것들 바꿔주기, 유저함수 삭제)
import re

for index, row in ngramdf.iterrows():
    if "setns" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("setns","308").split(",")
    if "time" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("time","201").split(",")
    if "capset" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("capset","126").split(",")
    if "fstat" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("fstat","5").split(",")
    if "uname" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("uname","63").split(",")
    if "__NR_vm86old" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("__NR_vm86old","436").split(",")
    if "__NR_socketcall" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("__NR_socketcall","437").split(",")
    if "sigaction" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("sigaction","13").split(",")
    if "ioctl" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("ioctl","16").split(",")
    if "kmalloc" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("kmalloc","12").split(",")
    if "shutdown" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("shutdown","48").split(",")
    if "msgrcv" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("msgrcv","70").split(",")
    ngramdf.iloc[index]['Ngram_syscall'] = re.findall(r'\d+', ",".join(row.Ngram_syscall)).copy()
ngramdf

,N,Ngram_syscall,num_of_exploit,EIDs
0,2,"[12, 41]",3,"[3, 50541, 926]"
1,2,"[257, 5]",40,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
2,2,"[107, 117]",3,"[1397, 40871, 47170]"
3,2,"[3, 103]",3,"[47168, 47169, 47170]"
4,2,"[8, 5]",4,"[12, 3, 43127, 50541]"
...,...,...,...,...
11350,63,"[248, 248, 41, 248, 248, 248, 54, 0, 3, 5, 9, ...",2,NaN
11351,63,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2,NaN
11352,64,"[14, 248, 248, 248, 41, 248, 248, 248, 54, 0, ...",2,NaN
11353,64,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2,NaN


In [8]:
import numpy as np
# Ngram 처리 후 변경 사항 처리
for index, row in ngramdf.iterrows():
    # Ngram_syscall 없는 애들은 NaN로 바꿔주기
    if not row.Ngram_syscall:
        ngramdf.iloc[index]['N'] = np.nan
    elif not isinstance(row.Ngram_syscall, list):
        ngramdf.iloc[index]['N'] = np.nan
    elif len(row.Ngram_syscall) == row.N:      
        ngramdf.iloc[index]['N'] = len(row.Ngram_syscall)

ngramdf = ngramdf.dropna(axis=0)
ngramdf

,N,Ngram_syscall,num_of_exploit,EIDs
0,2,"[12, 41]",3,"[3, 50541, 926]"
1,2,"[257, 5]",40,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
2,2,"[107, 117]",3,"[1397, 40871, 47170]"
3,2,"[3, 103]",3,"[47168, 47169, 47170]"
4,2,"[8, 5]",4,"[12, 3, 43127, 50541]"
...,...,...,...,...
11329,59,"[248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13, 14...",2,"[40871, 47170]"
11332,59,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
11335,60,"[248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13, 14...",2,"[40871, 47170]"
11336,60,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"


In [9]:
# pickle로 저장
import pickle

with open("defore_soft_dedup_ngramdf.pkl","wb") as f:
    pickle.dump(ngramdf, f)

In [1]:
# pickle load
import pickle

with open("defore_soft_dedup_ngramdf.pkl","rb") as f:
    ngramdf = pickle.load(f)

In [2]:
import subprocess
# 시스템콜 번호를 시스템콜 이름으로 바꾸기 위함 함수
def convert_num_name(sysnum):
    if sysnum == 436:
        return "vm86old"
    if sysnum == 437:
        return "socketcall"
    if sysnum == "clone":
        return "clone"
    if sysnum == "write":
        return "write"
    cmd = f'grep " {sysnum}$" /tmp/syscall_64.txt'
    sys_name = subprocess.check_output(cmd, shell=True).decode().strip().split()[1].replace("__NR_","")
    return sys_name

In [3]:
import subprocess
# 시스템콜 이름을 시스템콜 번호로 바꾸기 위함 함수
def convert_name_num(sysname):
    if sysname == "vm86old":
        return 436
    if sysname == "socketcall":
        return 437
    cmd = f'grep "\s{sysname}\s" /tmp/syscall_64.txt'
    try:
        sys_num = subprocess.check_output(cmd, shell=True).decode().strip().split()[0]
    except:
        cmd = f'grep "\s{sysname}" /tmp/syscall_64.txt'
        sys_num = subprocess.check_output(cmd, shell=True).decode().strip().split()[0]
    return sys_num

# Confine과 비교

### Confine Seccomp Profile 읽기

In [4]:
# Confine Git Repository Clone
import os
# os.system("git clone https://github.com/shamedgh/confine.git")

In [5]:
# Confine Seccomp profile 파일 명 및 파일 들고오기
import subprocess
cmd = "find confine/sample.results/ -name *.json"
json_files = subprocess.check_output(cmd, shell=True).decode().strip().split()
pwd = subprocess.check_output("pwd", shell=True).decode().strip()

# Json -> Dataframe
import pandas as pd
import json
from pandas import json_normalize

confine_profile = dict() # key: profile name, value: dataframe
for file in json_files:
    loc = pwd + "/" + file
    with open(loc, 'r', encoding='utf-8') as f:
        # value
        profile = json.load(f)
        nomalized_json = json_normalize(profile['syscalls'])
        value = nomalized_json.drop(['action','args'],axis='columns')
        # key
        remove_sentence = pwd+"/confine/sample.results/"
        key = loc.replace(remove_sentence,"").replace(".seccomp.json","")
        confine_profile[key] = value
        
len(confine_profile)

148

In [6]:
# Example
confine_profile["httpd"]

,name
0,pwrite64
1,readv
2,msync
3,mincore
4,pause
...,...
165,pwritev2
166,pkey_mprotect
167,pkey_alloc
168,pkey_free


In [11]:
# Dataframe 하나로 합치기
# Columns = {Applicaton Name, Seccomp Syscall, EID}
import pandas as pd
confine_seccomp = pd.DataFrame(columns=["appNm","seccomp_syscall","eid"])

for loc, appNm in enumerate(confine_profile):
    flat_seccomp = set()
    for syscall in confine_profile[appNm].name:
        flat_seccomp.add(syscall)
    confine_seccomp.loc[loc]=[appNm, flat_seccomp, set()]

confine_seccomp.head()

,appNm,seccomp_syscall,eid
0,celery,"{pkey_alloc, settimeofday, sched_setattr, fano...",{}
1,sematext-agent-monitoring-and-logging,"{sched_rr_get_interval, pkey_alloc, truncate, ...",{}
2,buildpack-deps,"{sched_rr_get_interval, pkey_alloc, truncate, ...",{}
3,elasticsearch,"{sched_rr_get_interval, pkey_alloc, lremovexat...",{}
4,jenkins,"{sched_rr_get_interval, pkey_alloc, truncate, ...",{}


In [ ]:
# system call name -> num 바꾸기
for index, row in confine_seccomp.iterrows():
    num_seccomp=set()
    for syscall in row.seccomp_syscall:
        sysnum = convert_name_num(syscall)
        num_seccomp.add(sysnum)
    confine_seccomp.loc[index, 'seccomp_syscall'] = num_seccomp

In [39]:
import pickle

# pickle로 저장
with open("confine_seccomp.pkl","wb") as f:
    pickle.dump(confine_seccomp, f)
    
confine_seccomp.head()

,appNm,seccomp_syscall,eid
0,celery,"{27, 159, 223, 154, 187, 29, 286, 246, 179, 27...",{}
1,sematext-agent-monitoring-and-logging,"{27, 159, 223, 154, 187, 29, 286, 246, 261, 17...",{}
2,buildpack-deps,"{27, 159, 223, 154, 187, 29, 286, 246, 261, 17...",{}
3,elasticsearch,"{27, 159, 223, 154, 187, 286, 246, 179, 272, 2...",{}
4,jenkins,"{159, 223, 154, 187, 286, 246, 179, 272, 237, ...",{}


In [15]:
# pickle load
import pickle

with open("confine_seccomp.pkl","rb") as f:
    confine_seccomp = pickle.load(f)

In [7]:
# Non-mitigable Exploit Code 조사

for index, row in confine_seccomp.iterrows():
    eids = set()
    for eid, paths in eid_all_path_dict.items():
        for path in paths:
            left = set(path) - row.seccomp_syscall
            if not left:
                eids.add(eid)
    confine_seccomp.loc[index, 'eid'] = eids
    
confine_seccomp.head()

,appNm,seccomp_syscall,eid
0,celery,"{329, 222, 314, 153, 152, 67, 321, 241, 159, 1...","{26131, 43345, 34923, 45010, 50135, 47170, 442..."
1,sematext-agent-monitoring-and-logging,"{329, 222, 314, 98, 264, 153, 152, 227, 67, 32...","{26131, 43127, 35161, 34923, 43345, 45010, 501..."
2,buildpack-deps,"{329, 222, 314, 264, 153, 152, 227, 67, 321, 2...","{26131, 43127, 35161, 34923, 43345, 45010, 501..."
3,elasticsearch,"{329, 222, 314, 153, 152, 227, 321, 241, 159, ...","{26131, 43127, 35161, 34923, 43345, 45010, 501..."
4,jenkins,"{329, 222, 314, 153, 152, 321, 241, 159, 187, ...","{26131, 43127, 43345, 34923, 45010, 50135, 471..."


In [8]:
import pickle
# confine_seccomp 저장
with open("confine_seccomp.pkl" ,"wb") as f:
    pickle.dump(confine_seccomp, f)

In [1]:
# pickle load
import pickle

with open("confine_seccomp.pkl","rb") as f:
    confine_seccomp = pickle.load(f)

In [17]:
for index, row in confine_seccomp.iterrows():
    if row.appNm == "nats-streaming":
        print(row.appNm, len(row.seccomp_syscall),len(row.eid))

nats-streaming 264 44


In [52]:
# 엑셀로 저장하기
confine_seccomp.to_excel("confine_seccomp.xlsx")

# N-gram pattern으로 mitigation 가능한 exploit 수 조사

In [18]:
confine_seccomp

,appNm,seccomp_syscall,eid
0,celery,"{312, 212, 279, 294, 251, 303, 317, 129, 226, ...","{12, 37722, 44298, 45010, 40871, 3, 43345, 261..."
1,sematext-agent-monitoring-and-logging,"{312, 212, 279, 294, 40, 251, 20, 303, 317, 12...","{12, 37722, 44298, 45010, 40871, 3, 43345, 349..."
2,buildpack-deps,"{312, 212, 279, 294, 40, 251, 20, 303, 317, 12...","{12, 37722, 44298, 45010, 40871, 3, 43345, 349..."
3,elasticsearch,"{312, 212, 279, 294, 251, 303, 317, 129, 226, ...","{12, 37722, 44298, 45010, 40871, 3, 43345, 349..."
4,jenkins,"{312, 212, 279, 251, 303, 317, 129, 226, 319, ...","{12, 37722, 44298, 45010, 40871, 3, 43345, 261..."
...,...,...,...
143,ibmjava,"{312, 212, 279, 294, 40, 251, 303, 317, 129, 2...","{12, 37722, 44298, 45010, 40871, 3, 43345, 349..."
144,softwareag-apigateway,"{312, 212, 279, 251, 303, 317, 129, 226, 319, ...","{37722, 44298, 45010, 40871, 43345, 26131, 349..."
145,piwik,"{312, 212, 279, 294, 251, 303, 317, 129, 226, ...","{12, 37722, 44298, 45010, 40871, 3, 43345, 349..."
146,amazonlinux,"{312, 212, 279, 294, 40, 251, 20, 303, 317, 12...","{12, 37722, 44298, 45010, 40871, 3, 43345, 349..."


In [19]:
len(confine_seccomp)

148

In [20]:
confine_mitigate = confine_seccomp[:]

empty = [set() for _ in range(0,len(confine_seccomp))]
confine_mitigate["mitigate"] = empty
confine_mitigate

/tmp/ipykernel_3900869/40943573.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  confine_mitigate["mitigate"] = empty


,appNm,seccomp_syscall,eid,mitigate
0,celery,"{312, 212, 279, 294, 251, 303, 317, 129, 226, ...","{12, 37722, 44298, 45010, 40871, 3, 43345, 261...",{}
1,sematext-agent-monitoring-and-logging,"{312, 212, 279, 294, 40, 251, 20, 303, 317, 12...","{12, 37722, 44298, 45010, 40871, 3, 43345, 349...",{}
2,buildpack-deps,"{312, 212, 279, 294, 40, 251, 20, 303, 317, 12...","{12, 37722, 44298, 45010, 40871, 3, 43345, 349...",{}
3,elasticsearch,"{312, 212, 279, 294, 251, 303, 317, 129, 226, ...","{12, 37722, 44298, 45010, 40871, 3, 43345, 349...",{}
4,jenkins,"{312, 212, 279, 251, 303, 317, 129, 226, 319, ...","{12, 37722, 44298, 45010, 40871, 3, 43345, 261...",{}
...,...,...,...,...
143,ibmjava,"{312, 212, 279, 294, 40, 251, 303, 317, 129, 2...","{12, 37722, 44298, 45010, 40871, 3, 43345, 349...",{}
144,softwareag-apigateway,"{312, 212, 279, 251, 303, 317, 129, 226, 319, ...","{37722, 44298, 45010, 40871, 43345, 26131, 349...",{}
145,piwik,"{312, 212, 279, 294, 251, 303, 317, 129, 226, ...","{12, 37722, 44298, 45010, 40871, 3, 43345, 349...",{}
146,amazonlinux,"{312, 212, 279, 294, 40, 251, 20, 303, 317, 12...","{12, 37722, 44298, 45010, 40871, 3, 43345, 349...",{}


In [4]:
# 길이 8 이상 조사 (for Cloud)
for i, rowi in confine_mitigate.iterrows():
    for eid in rowi.eid:
        for j, rowj in ngramdf.iterrows():
            if rowj.N >= 8 and eid in rowj.EIDs:
                confine_mitigate.iloc[i]['mitigate'].add(eid)
            
confine_mitigate        

,appNm,seccomp_syscall,eid,mitigate
0,celery,"{278, 27, 224, 245, 249, 290, 298, 242, 151, 3...","{43345, 40871, 34923, 41458, 50135, 3, 12, 471...","{43345, 40871, 41458, 3, 12, 17787, 47170, 450..."
1,sematext-agent-monitoring-and-logging,"{278, 27, 224, 141, 36, 264, 193, 249, 245, 29...","{43345, 43127, 44302, 40871, 34923, 1397, 4145...","{43127, 44302, 43345, 40871, 9083, 1397, 41458..."
2,buildpack-deps,"{278, 27, 224, 141, 36, 264, 245, 249, 290, 26...","{43345, 43127, 40871, 9083, 34923, 1397, 41458...","{43127, 9083, 40871, 43345, 1397, 41458, 35161..."
3,elasticsearch,"{278, 27, 224, 141, 100, 36, 245, 249, 290, 26...","{43345, 43127, 40871, 9083, 34923, 41458, 3516...","{43127, 9083, 40871, 43345, 41458, 35161, 3, 1..."
4,jenkins,"{278, 224, 245, 249, 290, 268, 298, 242, 151, ...","{43127, 43345, 40871, 9083, 34923, 41458, 5013...","{9083, 43127, 40871, 43345, 41458, 3, 12, 1778..."
...,...,...,...,...
143,ibmjava,"{278, 27, 224, 141, 36, 264, 245, 249, 290, 26...","{43345, 43127, 40871, 9083, 34923, 1397, 41458...","{43127, 9083, 40871, 43345, 1397, 41458, 35161..."
144,softwareag-apigateway,"{278, 224, 245, 36, 249, 290, 268, 298, 242, 1...","{43127, 43345, 40871, 9083, 34923, 41458, 5013...","{9083, 43127, 40871, 43345, 41458, 17787, 4717..."
145,piwik,"{278, 27, 224, 245, 36, 249, 290, 298, 242, 15...","{43345, 43127, 40871, 9083, 34923, 41458, 3516...","{43127, 9083, 40871, 43345, 41458, 35161, 3, 1..."
146,amazonlinux,"{278, 27, 224, 141, 100, 264, 36, 249, 245, 29...","{43345, 43127, 40871, 9083, 34923, 1397, 41458...","{43127, 9083, 40871, 43345, 1397, 41458, 35161..."


In [ ]:
import pickle
with open("confine_mitigate.pkl","wb") as f:
    pickle.dump(confine_mitigate,f)

In [ ]:
import pickle
with open("confine_mitigate.pkl","rb") as f:
    confine_mitigate = pickle.load(f)
confine_mitigate.tail()

In [ ]:
confine_mitigate.to_excel("confine_mitigate.xlsx")

In [ ]:
# Total exploit, mitigate 조사
total_seccomp_syscall = set()
total_eid = set()
total_mitigate = set()
for index, row in confine_mitigate.iterrows():
    total_seccomp_syscall = total_seccomp_syscall | row.seccomp_syscall
    total_eid = total_eid | row.eid
    total_mitigate = total_mitigate | row.mitigate
total = {"appNm" : "total", "seccomp_syscall":total_seccomp_syscall, "eid":total_eid, "mitigate":total_mitigate}
confine_mitigate.loc[index+1] = total

confine_mitigate.tail()

## For CCGRID (길이 5 이상 조사)

In [2]:
import pickle
with open("confine_mitigate.pkl","rb") as f:
    confine_mitigate_ccgrid = pickle.load(f)
confine_mitigate_ccgrid.tail()

,appNm,seccomp_syscall,eid,mitigate
143,ibmjava,"{220, 203, 101, 47, 207, 179, 253, 223, 287, 2...","{45010, 47170, 3, 43127, 41458, 12, 37722, 139...","{45010, 47170, 3, 43127, 41458, 12, 1397, 4087..."
144,softwareag-apigateway,"{220, 203, 207, 179, 223, 287, 230, 71, 242, 2...","{45010, 47170, 43127, 41458, 37722, 34923, 501...","{45010, 47170, 43127, 41458, 40871, 17787, 442..."
145,piwik,"{220, 203, 101, 47, 207, 179, 253, 223, 287, 2...","{45010, 47170, 3, 43127, 41458, 12, 37722, 349...","{45010, 47170, 3, 43127, 41458, 12, 40871, 351..."
146,amazonlinux,"{220, 203, 101, 47, 229, 207, 179, 253, 223, 2...","{45010, 47170, 3, 43127, 41458, 12, 37722, 139...","{45010, 47170, 3, 43127, 41458, 12, 1397, 4087..."
147,rabbitmq,"{220, 101, 207, 179, 253, 223, 287, 230, 71, 2...","{47170, 3, 43127, 41458, 12, 37722, 34923, 408...","{47170, 12, 3, 41458, 43127, 40871, 17787, 442..."


In [3]:
confine_mitigate_ccgrid.to_excel("confine_mitigate_ccgrid.xlsx")

In [3]:
# 길이 5 이상 조사 (for CCGrid)
for i, rowi in confine_mitigate_ccgrid.iterrows():
    for eid in rowi.eid:
        for j, rowj in ngramdf.iterrows():
            if rowj.N >= 5 and eid in rowj.EIDs:
                confine_mitigate_ccgrid.iloc[i]['mitigate'].add(eid)
            
confine_mitigate_ccgrid

,appNm,seccomp_syscall,eid,mitigate
0,celery,"{220, 101, 207, 179, 253, 223, 287, 230, 71, 2...","{45010, 47170, 3, 12, 41458, 37722, 34923, 408...","{45010, 47170, 3, 12, 41458, 40871, 17787, 442..."
1,sematext-agent-monitoring-and-logging,"{220, 203, 101, 229, 19, 179, 207, 253, 223, 2...","{45010, 47170, 3, 43127, 44302, 41458, 12, 139...","{45010, 47170, 3, 43127, 44302, 41458, 12, 139..."
2,buildpack-deps,"{220, 203, 101, 47, 229, 207, 179, 253, 223, 2...","{45010, 47170, 3, 43127, 41458, 12, 37722, 139...","{45010, 47170, 3, 43127, 41458, 12, 1397, 4087..."
3,elasticsearch,"{220, 203, 101, 47, 207, 179, 253, 223, 287, 2...","{45010, 47170, 3, 43127, 41458, 12, 37722, 349...","{45010, 47170, 3, 43127, 41458, 12, 40871, 351..."
4,jenkins,"{220, 203, 101, 207, 179, 223, 287, 230, 71, 2...","{45010, 47170, 3, 43127, 41458, 12, 37722, 349...","{45010, 47170, 3, 43127, 41458, 12, 40871, 177..."
...,...,...,...,...
143,ibmjava,"{220, 203, 101, 47, 207, 179, 253, 223, 287, 2...","{45010, 47170, 3, 43127, 41458, 12, 37722, 139...","{45010, 47170, 3, 43127, 41458, 12, 1397, 4087..."
144,softwareag-apigateway,"{220, 203, 207, 179, 223, 287, 230, 71, 242, 2...","{45010, 47170, 43127, 41458, 37722, 34923, 501...","{45010, 47170, 43127, 41458, 40871, 17787, 442..."
145,piwik,"{220, 203, 101, 47, 207, 179, 253, 223, 287, 2...","{45010, 47170, 3, 43127, 41458, 12, 37722, 349...","{45010, 47170, 3, 43127, 41458, 12, 40871, 351..."
146,amazonlinux,"{220, 203, 101, 47, 229, 207, 179, 253, 223, 2...","{45010, 47170, 3, 43127, 41458, 12, 37722, 139...","{45010, 47170, 3, 43127, 41458, 12, 1397, 4087..."


In [4]:
import pickle
with open("confine_mitigate_ccgrid.pkl","wb") as f:
    pickle.dump(confine_mitigate_ccgrid,f)

In [5]:
import pickle
with open("confine_mitigate_ccgrid.pkl","rb") as f:
    confine_mitigate_ccgrid = pickle.load(f)
confine_mitigate_ccgrid.tail()

,appNm,seccomp_syscall,eid,mitigate
143,ibmjava,"{101, 220, 203, 47, 207, 179, 253, 223, 287, 2...","{9083, 47170, 3, 43127, 41458, 12, 37722, 1397...","{47170, 3, 43127, 41458, 12, 1397, 40871, 3516..."
144,softwareag-apigateway,"{220, 203, 207, 179, 223, 287, 230, 71, 242, 2...","{9083, 47170, 43127, 37722, 41458, 34923, 4087...","{47170, 43127, 41458, 40871, 17787, 44298, 450..."
145,piwik,"{101, 220, 203, 47, 207, 179, 253, 223, 287, 2...","{9083, 47170, 3, 43127, 41458, 12, 37722, 3492...","{47170, 3, 43127, 41458, 12, 40871, 35161, 177..."
146,amazonlinux,"{101, 220, 203, 47, 229, 207, 179, 253, 223, 2...","{9083, 47170, 3, 43127, 41458, 12, 37722, 1397...","{47170, 3, 43127, 41458, 12, 1397, 40871, 3516..."
147,rabbitmq,"{101, 220, 207, 179, 253, 223, 287, 230, 71, 2...","{47170, 12, 3, 41458, 43127, 37722, 34923, 408...","{47170, 3, 12, 41458, 43127, 40871, 17787, 442..."


In [6]:
# Total exploit, mitigate 조사
total_seccomp_syscall = set()
total_eid = set()
total_mitigate = set()
for index, row in confine_mitigate_ccgrid.iterrows():
    total_seccomp_syscall = total_seccomp_syscall | row.seccomp_syscall
    total_eid = total_eid | row.eid
    total_mitigate = total_mitigate | row.mitigate
total = {"appNm" : "total", "seccomp_syscall":total_seccomp_syscall, "eid":total_eid, "mitigate":total_mitigate}
confine_mitigate_ccgrid.loc[index+1] = total

confine_mitigate_ccgrid.tail()

,appNm,seccomp_syscall,eid,mitigate
144,softwareag-apigateway,"{220, 203, 207, 179, 223, 287, 230, 71, 242, 2...","{9083, 47170, 43127, 37722, 41458, 34923, 4087...","{47170, 43127, 41458, 40871, 17787, 44298, 450..."
145,piwik,"{101, 220, 203, 47, 207, 179, 253, 223, 287, 2...","{9083, 47170, 3, 43127, 41458, 12, 37722, 3492...","{47170, 3, 43127, 41458, 12, 40871, 35161, 177..."
146,amazonlinux,"{101, 220, 203, 47, 229, 207, 179, 253, 223, 2...","{9083, 47170, 3, 43127, 41458, 12, 37722, 1397...","{47170, 3, 43127, 41458, 12, 1397, 40871, 3516..."
147,rabbitmq,"{101, 220, 207, 179, 253, 223, 287, 230, 71, 2...","{47170, 12, 3, 41458, 43127, 37722, 34923, 408...","{47170, 3, 12, 41458, 43127, 40871, 17787, 442..."
148,total,"{101, 47, 229, 207, 179, 19, 287, 230, 71, 148...","{38390, 18411, 33336, 47170, 12, 44303, 43127,...","{38390, 18411, 33336, 47170, 12, 44303, 43127,..."


In [7]:
confine_mitigate_ccgrid.to_excel("confine_mitigate_ccgrid.xlsx")